# Sentiment Analysis Project using Machine Learning
This project performs sentiment analysis on text data using NLP and machine learning techniques.

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## 2. Load Dataset

In [ ]:
# Sample dataset from NLTK movie reviews (or replace with your dataset)
from sklearn.datasets import fetch_20newsgroups
data = fetch_20newsgroups(subset='all', categories=['rec.sport.baseball', 'sci.med'], shuffle=True, random_state=42)
df = pd.DataFrame({'text': data.data, 'label': data.target})
df['label'] = df['label'].map({0: 'Negative', 1: 'Positive'})
df.head()

## 3. Preprocess Text

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

df['clean_text'] = df['text'].apply(preprocess_text)
df[['text', 'clean_text']].head()

## 4. Vectorization

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['clean_text']).toarray()
y = df['label']

## 5. Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 6. Train Model (Logistic Regression)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

## 7. Evaluation

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.show()

## 8. Predict Custom Sentences

In [ ]:
def predict_sentiment(text):
    cleaned = preprocess_text(text)
    vector = tfidf.transform([cleaned]).toarray()
    prediction = model.predict(vector)
    return prediction[0]

sample = "The new medicine has helped me recover fast."
print("Prediction:", predict_sentiment(sample))